# Dataset Index Generation
Generates indices for train, validation and test sets

In [1]:
import h5py
from progressbar import *
import re
import numpy as np

## Options

In [2]:
n_test_files = 400
n_val_files = 100
labels = (0, 1)

## Load dataset

In [3]:
data_path = "/scratch/jgao/data/HKHybrid/all_w_gamma.hdf5"
f = h5py.File(data_path, "r")

OSError: Unable to open file (bad object header version number)

In [ ]:
event_labels = np.array(f['labels'])
root_files = np.array(f['root_files']).astype(str)
print(len(event_labels))

## Find the files of each label and indices of each file

In [ ]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
files_in_labels = {l: sorted(set(root_files[event_labels==l]), key=natural_keys) for l in labels}
idxs_in_files = {f: range(i, i+c) for f,i,c in zip(*np.unique(root_files, return_index=True, return_counts=True))}

In [ ]:
for l, f in files_in_labels.items():
    print("label", l,"has", len(f),"files and ", sum([len(idxs_in_files[i]) for i in f]), "indices")

## Create the splits

In [ ]:
split_files = {"test_idxs":  [f for l in labels for f in files_in_labels[l][:n_test_files]],
               "val_idxs":   [f for l in labels for f in files_in_labels[l][n_test_files:n_test_files+n_val_files]],
               "train_idxs": [f for l in labels for f in files_in_labels[l][n_test_files+n_val_files:]]}
split_idxs = {k: [i for f in v for i in idxs_in_files[f]] for k, v in split_files.items()}

In [ ]:
for s in split_files.keys():
    print(s,"has", len(split_files[s]),"files and", len(split_idxs[s]),"indices")

In [ ]:
# Verify that all events are uniquely accounted for
all_indices = np.concatenate(list(split_idxs.values()))
print(len(event_labels))
print(len(all_indices))
print(len(set(all_indices)))

## Save file

In [18]:
np.savez('/scratch/jgao/data/HKHybrid/e-gamma_idxs.npz', **split_idxs)